In [21]:
FONT_LINKS = """
https://web.archive.org/web/20130404225932/http://www.iol.ie/~sob/gadelica.zip
http://web.archive.org/web/20120830224624/http://www.gaelchlo.com/bunardgc.zip
http://web.archive.org/web/20220322030900/https://www.gaelchlo.com/bunargc.zip
http://web.archive.org/web/20120830224559/http://www.gaelchlo.com/bunartgc.zip
http://web.archive.org/web/20191216105737/http://www.gaelchlo.com/bundgc.zip
http://web.archive.org/web/20120830224343/http://www.gaelchlo.com/bunnod.zip
http://web.archive.org/web/20191218121436/http://www.gaelchlo.com/buntgc.zip
http://web.archive.org/web/20160408212120/http://gaelchlo.com/seandgc.zip
http://web.archive.org/web/20160408212013/http://gaelchlo.com/seantgc.zip
http://web.archive.org/web/20220322030900/https://www.gaelchlo.com/bunargc.zip
http://web.archive.org/web/20240119141814/https://www.gaelchlo.com/bungc.zip
http://web.archive.org/web/20160408212120/http://gaelchlo.com/seandgc.zip
http://web.archive.org/web/20240119143113/https://www.gaelchlo.com/seangc.zip
http://web.archive.org/web/20160408212013/http://gaelchlo.com/seantgc.zip
http://web.archive.org/web/20240619180031/http://www.gaelchlo.com/urgc.zip
http://web.archive.org/web/20240119141944/https://www.gaelchlo.com/aongc.zip
http://web.archive.org/web/20240119141941/https://www.gaelchlo.com/aonw.zip
http://web.archive.org/web/20240619182847/https://www.gaelchlo.com/ardeorgc.zip
http://web.archive.org/web/20240119142022/https://www.gaelchlo.com/ardeorw.zip
http://web.archive.org/web/20240119142032/https://www.gaelchlo.com/ardgc.zip
http://web.archive.org/web/20240119142031/https://www.gaelchlo.com/ardw.zip
http://web.archive.org/web/20240119142040/https://www.gaelchlo.com/barrgc.zip
http://web.archive.org/web/20240119142038/https://www.gaelchlo.com/barrw.zip
http://web.archive.org/web/20240119141814/https://www.gaelchlo.com/bungc.zip
http://web.archive.org/web/20240119141805/https://www.gaelchlo.com/bunw.zip
http://web.archive.org/web/20240119142035/https://www.gaelchlo.com/casgc.zip
http://web.archive.org/web/20240119142033/https://www.gaelchlo.com/casw.zip
http://web.archive.org/web/20240619182918/https://www.gaelchlo.com/corrgc.zip
http://web.archive.org/web/20240619182955/https://www.gaelchlo.com/corrw.zip
http://web.archive.org/web/20240119142042/https://www.gaelchlo.com/deasgc.zip
http://web.archive.org/web/20240119142041/https://www.gaelchlo.com/deasw.zip
http://web.archive.org/web/20240119142045/https://www.gaelchlo.com/dluthgc.zip
http://web.archive.org/web/20240119142046/https://www.gaelchlo.com/dluthw.zip
http://web.archive.org/web/20240119142026/https://www.gaelchlo.com/dubhgc.zip
http://web.archive.org/web/20240119142027/https://www.gaelchlo.com/dubhw.zip
http://web.archive.org/web/20240119142025/https://www.gaelchlo.com/fiorgc.zip
http://web.archive.org/web/20240119142023/https://www.gaelchlo.com/fiorw.zip
http://web.archive.org/web/20240119142051/https://www.gaelchlo.com/geargc.zip
http://web.archive.org/web/20240119142050/https://www.gaelchlo.com/gearw.zip
http://web.archive.org/web/20240119142055/https://www.gaelchlo.com/glangc.zip
http://web.archive.org/web/20240119142053/https://www.gaelchlo.com/glanw.zip
http://web.archive.org/web/20240119143123/https://www.gaelchlo.com/langc.zip
http://web.archive.org/web/20240119143128/https://www.gaelchlo.com/lanw.zip
http://web.archive.org/web/20240619183031/https://www.gaelchlo.com/lomgc.zip
http://web.archive.org/web/20240619183107/https://www.gaelchlo.com/meargc.zip
http://web.archive.org/web/20240619183116/https://www.gaelchlo.com/mearw.zip
http://web.archive.org/web/20240619183125/https://www.gaelchlo.com/mingc.zip
"""

In [22]:
FONTS = [x for x in FONT_LINKS.split("\n") if x != ""]

In [ ]:
for font in FONTS:
  !wget {font}
  !unzip {font.split("/")[-1]}

In [11]:
!mkdir fonts
!mv *.ttf fonts

In [ ]:
%%capture
!apt install python3-fontforge

In [ ]:
%%capture
!apt install lynx

In [40]:
!lynx -dump https://www.gaelchlo.com/clonna1.html|grep html|grep '[12][0-9]\.'|awk '{print $2}'|while read i;do lynx -dump $i|grep zip;done|grep https|awk '{print $2}'|sort|uniq > fontlist

In [ ]:
!wget -i fontlist

In [ ]:
%%capture
!for i in *zip;do unzip -o $i;done

In [46]:
!cat fontlist|sort|uniq|awk '{print "http://web.archive.org/web/" $0}' > iafonts
!wget -i iafonts -o iafonts.log

In [45]:
from pathlib import Path
import fontforge

for otf in Path(".").glob("**/*.otf"):
    font = fontforge.open(str(otf))
    outname = "fonts/" + otf.stem + ".ttf"
    font.save(outname)

In [50]:
!grep '^-' iafonts.log|grep 'web/2'|awk '{print $NF}' > iafonts-arc

In [51]:
!wc -l iafonts*

   38 iafonts
   33 iafonts-arc
  545 iafonts.log
  616 total
